## Terminology - Project
Authors: Cécile MACAIRE & Ludivine ROBERT 

## Librairies 

In [12]:
import pandas as pd
import spacy
spacy_nlp = spacy.load('en_core_web_sm')

## Code 

In [2]:
# Read data from lexicon and store in dataframe
def read_data(file):
    """Read data file with pandas dataframe"""
    return pd.read_csv(file, sep='\t')
  
def select_data(dataframe):
    """We keep only columns pattern, pilot and lemma"""
    return dataframe[['pattern', 'pilot', 'lemma']]

In [85]:
def read_file(file):
    with open(file, 'r') as f:
        return f.read()
        
def lemma_posttag(file):
    text = read_file(file)
    doc = spacy_nlp(text)
    tokens = [token.text for token in doc]
    new_pos = []
    pos = []
    lemma = []
    t = []
    for token in doc:
        t.append(token.text)
        lemma.append(token.lemma_)
        pos.append(token.pos_)
        if token.pos_ == 'NOUN':
            new_pos.append('N')
        elif token.pos_ == 'VERB':
            new_pos.append('V')
        elif token.pos_ == 'ADJ':
            new_pos.append('A')
        elif token.pos_ == 'CCONJ' or token.pos_ == 'SCONJ':
            new_pos.append('C')
        elif token.pos_ == 'PART' or token.pos_ == 'ADP':
            new_pos.append('P')
        else:
            new_pos.append('')
    frame = pd.DataFrame({'tokens':t, 'lemma':lemma, 'pos':pos, 'new_pos':new_pos})
    return frame

In [86]:
# Link terms with text
def annotate(terms_dataframe, text_dataframe):
    for i, token in enumerate(text_dataframe['lemma']):
        for term in terms_dataframe['lemma']:
            if token == term:
                text_dataframe['tokens'][i] = '['+text_dataframe['tokens'][i]+']'
    return text_dataframe

In [87]:
def rules(lemma, pattern):
    """Define rules for a lemma according to its pattern"""

In [88]:
if __name__== "__main__":
    init_data = read_data('tts-lexicon.tsv')
    data = select_data(init_data)
    text_dataframe = lemma_posttag('test.txt')
    print(annotate(data, text_dataframe))

          tokens        lemma    pos new_pos
0    INTERSPEECH  INTERSPEECH  PROPN        
1           2013         2013    NUM        
2             \n           \n  SPACE        
3         TUNDRA       TUNDRA  PROPN        
4              :            :  PUNCT        
..           ...          ...    ...     ...
279            -            -  PUNCT        
280     [speech]       speech   NOUN       N
281            ,            ,  PUNCT        
282    audiobook    audiobook  PROPN        
283         data         data  PROPN        

[284 rows x 4 columns]
